In [58]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import Model

In [59]:
#Importing the data from the directory
train_datagen= ImageDataGenerator()      
test_datagen = ImageDataGenerator()      
train_generator = train_datagen.flow_from_directory(
        'datasetjpgrgb/train',
        target_size=(150, 150),
        color_mode='rgb',
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'datasetjpgrgb/val',
        target_size=(150, 150),
        color_mode='rgb',
        batch_size=32,
        class_mode='categorical')

Found 30000 images belonging to 3 classes.
Found 7500 images belonging to 3 classes.


In [60]:
preprocess_input=tf.keras.applications.resnet.preprocess_input     #Preprocessing the inputs for the ResNet

In [61]:
base_model = tf.keras.applications.resnet50.ResNet50(input_shape=(150,150,3),  #Loading The ResNet Model
                                               include_top=False,
                                               weights='imagenet')

In [62]:
base_model.trainable=False                                         #Keeping the training false 

In [63]:
base_model.summary()                                               #The Summary of the model

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [64]:
last_layer = base_model.get_layer("conv5_block3_out")            #Getting the last layer of the ResNet
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output


last layer output shape:  (None, 5, 5, 2048)


In [65]:
#The input to the model
inputs=tf.keras.layers.Input(shape=(150,150,3))
a=preprocess_input(inputs)
last_output=base_model(a,training=False)

In [66]:
#This model ends with a convolutional layer of size(,5,5,2048)
#if directly feed to a dense layer of 1024 activations the number of parameters will be over 52 Million
#These parameters will cause slow training.
#To reduce this problem a pooling layer with size of 5X5 is used so as to reduce the dimension to 209K
y=tf.keras.layers.GlobalAveragePooling2D()(last_output)
# Flatten the output layer to 1 dimension
x = tf.keras.layers.Flatten()(y)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = tf.keras.layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = tf.keras.layers.Dropout(0.2)(x)                  



In [67]:
# Add a final sigmoid layer for classification
prediction_layer = tf.keras.layers.Dense(3, activation='softmax')           
outputs=prediction_layer(x)

In [68]:
model=tf.keras.Model(inputs,outputs)

In [69]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.95,
    beta_2=0.999,
    epsilon=1e-07
),
    loss='CategoricalCrossentropy',
    metrics=['AUC',"accuracy"]
)

In [70]:
model.summary()

Model: "functional_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 150, 150, 3)]     0         
_________________________________________________________________
tf_op_layer_strided_slice_6  [(None, 150, 150, 3)]     0         
_________________________________________________________________
tf_op_layer_BiasAdd_6 (Tenso [(None, 150, 150, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 5, 5, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_13  (None, 2048)              0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_25 (Dense)             (None, 1024)            

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=256,
        epochs=500,
        validation_data=validation_generator,
        validation_steps=100)

Epoch 1/500
256/256 [==============================] - 33s 129ms/step - loss: 1.2763 - auc: 0.5405 - accuracy: 0.3737 - val_loss: 1.0534 - val_auc: 0.6322 - val_accuracy: 0.4366
Epoch 2/500
256/256 [==============================] - 32s 126ms/step - loss: 1.0539 - auc: 0.6214 - accuracy: 0.4294 - val_loss: 1.0252 - val_auc: 0.6612 - val_accuracy: 0.4597
Epoch 3/500
256/256 [==============================] - 32s 126ms/step - loss: 1.0495 - auc: 0.6243 - accuracy: 0.4265 - val_loss: 1.0465 - val_auc: 0.6310 - val_accuracy: 0.4263
Epoch 4/500
256/256 [==============================] - 32s 126ms/step - loss: 1.0408 - auc: 0.6363 - accuracy: 0.4349 - val_loss: 1.0222 - val_auc: 0.6709 - val_accuracy: 0.4688
Epoch 5/500
256/256 [==============================] - 32s 126ms/step - loss: 1.0393 - auc: 0.6356 - accuracy: 0.4415 - val_loss: 1.0338 - val_auc: 0.6483 - val_accuracy: 0.4494
Epoch 6/500
256/256 [==============================] - 32s 126ms/step - loss: 1.0325 - auc: 0.6442 - accuracy: